In [2]:
from pyspark.sql import SparkSession, Row, functions


---

# Create Spark Session
- SparkSession is the entry point to Spark SQL. It is one of the very first objects you create while developing a Spark SQL application

In [ ]:
spark = SparkSession.builder.appName("basic").getOrCreate()
spark

---

# Read data

In [86]:
df_pyspark = spark.read.csv("../data/basic1.csv", header=True, inferSchema=True)
df_pyspark

DataFrame[Name: string, age: int, Experience: int, Salary: int]

# Show schema

In [14]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



# Get columns's name

In [16]:
df_pyspark.columns

['Name', 'age', 'Experience', 'Salary']

# SELECT name, experience FROM data

In [25]:
df_pyspark.select(["Name", "Experience"]).show()

# Other way
# df_pyspark[["Name", "Salary"]].show()


+---------+------+
|     Name|Salary|
+---------+------+
|    Krish| 30000|
|Sudhanshu| 25000|
|    Sunny| 20000|
|     Paul| 20000|
|   Harsha| 15000|
|  Shubham| 18000|
+---------+------+



# Add columns in Pyspark dataframe

In [29]:
df_pyspark = df_pyspark.withColumn("Experience after 2 years", df_pyspark["Experience"] + 2)
df_pyspark.show()

+---------+---+----------+------+------------------------+
|     Name|age|Experience|Salary|Experience after 2 years|
+---------+---+----------+------+------------------------+
|    Krish| 31|        10| 30000|                      12|
|Sudhanshu| 30|         8| 25000|                      10|
|    Sunny| 29|         4| 20000|                       6|
|     Paul| 24|         3| 20000|                       5|
|   Harsha| 21|         1| 15000|                       3|
|  Shubham| 23|         2| 18000|                       4|
+---------+---+----------+------+------------------------+



# Drop Column

In [32]:
df_pyspark = df_pyspark.drop("Experience after 2 years")
df_pyspark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



---

# Handling null values

In [33]:
df2 = spark.read.csv("../data/null_example.csv", header=True, inferSchema=True)
df2.show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8|  NULL|
|    Sunny|  29|         4| 20000|
|     Paul|NULL|         3| 20000|
|     NULL|  21|         1| 15000|
|  Shubham|  23|      NULL| 18000|
+---------+----+----------+------+



### - Drop any row that contains NULL

In [38]:
df2.na.drop(how="any").show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
|Krish| 31|        10| 30000|
|Sunny| 29|         4| 20000|
+-----+---+----------+------+



- ### Drop rows that contain NULL in 'Name' column

In [39]:
df2.na.drop(how="any",subset="Name").show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8|  NULL|
|    Sunny|  29|         4| 20000|
|     Paul|NULL|         3| 20000|
|  Shubham|  23|      NULL| 18000|
+---------+----+----------+------+



---

# Filter Operations

### - Filter Salary <= 20000

In [45]:
df_pyspark.filter(df_pyspark["Salary"] <= 20000).show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



Select the name and salary column only

In [46]:
df_pyspark.filter(df_pyspark["Salary"] <= 20000).select(["Name", "Salary"]).show()

+-------+------+
|   Name|Salary|
+-------+------+
|  Sunny| 20000|
|   Paul| 20000|
| Harsha| 15000|
|Shubham| 18000|
+-------+------+



### - Filter with multiple conditions
- AND:          '&'
- OR:       '|'
- NOT:      '~'

In [50]:
df_pyspark.filter((df_pyspark["Salary"] <= 20000) & 
                (df_pyspark["age"] < 25)).select(["Name", "Salary", "age"]).show()

+-------+------+---+
|   Name|Salary|age|
+-------+------+---+
|   Paul| 20000| 24|
| Harsha| 15000| 21|
|Shubham| 18000| 23|
+-------+------+---+



---

In [57]:
df3 = spark.read.csv("../data/basic2.csv", header=True, inferSchema=True)
df3.show()

+---------+------------+------+
|     Name| Departments|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



# Group By

## - Find maximum salary

In [61]:
df3.groupBy("Name")

GroupedData[grouping expressions: [Name], value: [Name: string, Departments: string ... 1 more field], type: GroupBy]

- groupBy("Name") return a grouped data, then we apply it with an aggregate function 'max'

In [82]:
total_salary = df3.groupBy("Name").max()
total_salary.show()

+---------+-----------+
|     Name|max(salary)|
+---------+-----------+
|Sudhanshu|      20000|
|    Sunny|      10000|
|    Krish|      10000|
|   Mahesh|       4000|
+---------+-----------+



-> You can't sum the string, so the sum only applied to the 'Salary' column <br>
-> Now we can find the maximum salary

## - Find the department that give the most salary

In [84]:
df3.groupBy("Departments").sum().show()

+------------+-----------+
| Departments|sum(salary)|
+------------+-----------+
|         IOT|      15000|
|    Big Data|      15000|
|Data Science|      43000|
+------------+-----------+



- How about the average salary?

In [73]:
df3.groupBy("Departments").avg().show()

+------------+-----------+
| Departments|avg(salary)|
+------------+-----------+
|         IOT|     7500.0|
|    Big Data|     3750.0|
|Data Science|    10750.0|
+------------+-----------+



## - Count the departments

In [80]:
df3.groupBy("Departments").count().show()

+------------+-----+
| Departments|count|
+------------+-----+
|         IOT|    2|
|    Big Data|    4|
|Data Science|    4|
+------------+-----+



## - Apply aggregate function directly

In [85]:
df3.agg({"Salary":"max"}).show()

+-----------+
|max(Salary)|
+-----------+
|      20000|
+-----------+

